# Seras学院 データ解析 (Data Analysis)

Google Sheets (`occupancy_logs`, `open_logs`) からデータを読み込みます。

In [1]:
import os
import gspread
import polars as pl
from google.oauth2.service_account import Credentials
from dotenv import load_dotenv

load_dotenv()

# ---------------------------------------------------------
# Google Auth & Data Loading
# ---------------------------------------------------------
credentials_config = {
    "type": "service_account",
    "client_email": os.getenv('GOOGLE_SERVICE_ACCOUNT_EMAIL'),
    "private_key": os.getenv('GOOGLE_PRIVATE_KEY').replace('\\n', '\n') if os.getenv('GOOGLE_PRIVATE_KEY') else None,
    "token_uri": "https://oauth2.googleapis.com/token",
}

creds = Credentials.from_service_account_info(
    credentials_config, 
    scopes=['https://www.googleapis.com/auth/spreadsheets']
)
gc = gspread.authorize(creds)
sh = gc.open_by_key(os.getenv('OCCUPANCY_SPREADSHEET_ID'))

# Load Spreadsheets to Polars DataFrame
df_occupancy = pl.DataFrame(sh.worksheet('occupancy_logs').get_all_records())
df_open = pl.DataFrame(sh.worksheet('open_logs').get_all_records())

print(f"Loaded occupancy_logs: {df_occupancy.shape}")
print(f"Loaded open_logs: {df_open.shape}")

Loaded occupancy_logs: (408, 7)
Loaded open_logs: (6, 5)


In [2]:
display(df_occupancy.head())
display(df_open.head())

Timestamp,Date,Day,Hour,Building1,Building2,Total
str,str,str,i64,i64,i64,i64
"""2025/12/05 6:00:15""","""2025/12/05""","""Fri""",6,0,0,0
"""2025/12/05 6:15:15""","""2025/12/05""","""Fri""",6,0,0,0
"""2025/12/05 6:30:15""","""2025/12/05""","""Fri""",6,0,0,0
"""2025/12/05 6:45:14""","""2025/12/05""","""Fri""",6,0,0,0
"""2025/12/05 7:00:18""","""2025/12/05""","""Fri""",7,0,0,0


Timestamp,Actor Name,Action,Building,UserAgent / Context
str,str,str,str,str
"""2025/12/10 13:26:12""","""わたひき""","""OPEN""","""2号館""","""Web Portal"""
"""2025/12/10 13:26:31""","""わたひき""","""CLOSE""","""2号館""","""Web Portal"""
"""2025/12/10 13:43:00""","""わたひき""","""OPEN""","""2号館""","""Web Portal"""
"""2025/12/11 0:39:42""","""わたひき""","""CLOSE""","""2号館""","""Web Portal"""
"""2025/12/11 0:51:06""","""わたひき""","""OPEN""","""2号館""","""Web Portal"""
